# Texas Power Outage Visualization

**Description**: 

I want to visualize the Texas power outage during Feb.10 to Feb.19 hour by hour in Tableau.
* Firstly, collecting outage data, weather history data and Texas county data.
* Secondly, processing the data.
* Lastly, visualizing the processed data in Tableau or visualizing data by Plotly in Notebook.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)
from tqdm import tqdm
from datetime import datetime
from meteostat import Point, Hourly, Daily, Stations

Populating the interactive namespace from numpy and matplotlib


# 1 Data Collection

In this part, I will collect outage data, weather history data and Texas county data.
- Outage data is from: [ERCOT, Generation Resource and Energy Storage Resource Outages and Derates for February 10-19, 2021 Excel Version](http://www.ercot.com/content/wcm/lists/226521/Unit_Outage_Data_20210312.xlsx).
- Texas county list is from: [Texas-Counties-Centroid-Map](https://data.texas.gov/dataset/Texas-Counties-Centroid-Map/ups3-9e8m/data). Another source is [Wikipedia](https://en.wikipedia.org/wiki/User:Michael_J/County_table).
- Weather history data is from: [Meteostat](https://dev.meteostat.net/python/), weather stations list is [here](https://github.com/meteostat/weather-stations).

## 1.1 Outage data

In [2]:
outage = pd.read_excel('outage_feb1019.xlsx', sheet_name='OutageData')
print(f'Outage size: {outage.shape}')
outage.head(3)

Outage size: (2599, 11)


,STATION,STATION LONG NAME,UNIT NAME,SEASONAL MAX MW (HSL),AVAILABLE MW AFTER OUTAGE/DERATE,MW REDUCTION FROM OUTAGE/DERATE,FUEL TYPE,START,END,RESOURCE ENTITY,COUNTY
0,NBOHR,NIELS BOHR,UNIT1,197,20,177,WIND,2021-02-10 00:00:00,2021-02-12 04:20:00,BEARKAT WIND ENERGY I LLC (RE),GLASSCOCK
1,KEECHI,KEECHI WIND,U1,110,0,110,WIND,2021-02-10 00:30:00,2021-02-15 08:51:00,KEECHI WIND LLC (RE),JACK
2,BLSUMIT3,BLUE SUMMIT 3,UNIT_17,13,1,12,WIND,2021-02-10 01:15:00,2021-02-10 18:46:00,BLUE SUMMIT III WIND LLC (RE),HARDEMAN


In [4]:
outage['FUEL TYPE'].unique()

array(['WIND', 'NG', 'COAL', 'SOLAR', 'ESR', 'HYDRO', 'NUCLEAR'],
      dtype=object)

## 1.2 County data

In [5]:
cnty = pd.read_csv('Texas_Counties_Centroid_Map.csv')
print(f'County size: {cnty.shape}')
cnty.head(3)

County size: (254, 8)


,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"


## 1.3 Weather history data

In [ ]:
###scrape weather history data hourly
# stations = Stations()
# # Set time period
# start = datetime(2021, 2, 10)
# end = datetime(2021, 2, 19, 23, 59)

# # for idx in tqdm(range(len(cnty))):
# # for idx in range(len(cnty)):
# idx_s = 200 #change this in tiny step to avoid blocking
# idx_e = len(cnty)
# for idx in tqdm(range(idx_s, idx_e)):
#     cnty_name = cnty['CNTY_NM'][idx]
#     lat = cnty['X (Lat)'][idx].round(4)
#     lon = cnty['Y (Long)'][idx].round(4)
#     stations = stations.nearby(lon, lat)
#     station = stations.fetch(1)
    
#     # Get daily data for 2018
#     data = Hourly(station.index[0], start, end) #https://dev.meteostat.net/python/hourly.html#example
#     data = data.fetch()
#     data.reset_index(inplace=True) #the time was used as index, reset it
#     data['COUNTY'] = cnty_name
    
#     if idx == idx_s:
#         hourly_data = data
#     else:
#         hourly_data = hourly_data.append(data)
        
        
# hourly_data.reset_index(inplace=True)
# hourly_data.to_csv('hourly_data_200_last.csv')

In [6]:
hourly_data_0_99 = pd.read_csv('hourly_data_0_99.csv')
hourly_data_100_149 = pd.read_csv('hourly_data_100_149.csv')
hourly_data_150_199 = pd.read_csv('hourly_data_150_199.csv')
hourly_data_200_last = pd.read_csv('hourly_data_200_last.csv')

hourly_data = pd.concat([hourly_data_0_99, hourly_data_100_149, hourly_data_150_199, hourly_data_200_last], axis=0)
hourly_data.head(3)

,Unnamed: 0,index,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,COUNTY
0,0,0,2021-02-10 00:00:00,13.0,10.0,82.0,0.0,NaN,88.0,5.0,NaN,1015.1,NaN,NaN,Gonzales
1,1,1,2021-02-10 01:00:00,13.0,9.1,77.0,0.0,NaN,40.0,13.3,NaN,1015.6,NaN,NaN,Gonzales
2,2,2,2021-02-10 02:00:00,13.0,9.1,77.0,0.0,NaN,60.0,15.8,NaN,1016.1,NaN,NaN,Gonzales


In [7]:
hourly_data.drop(['Unnamed: 0', 'index'], inplace=True, axis=1)
hourly_data.rename(columns={'COUNTY': 'CNTY_NM'}, inplace=True)
hourly_data.reset_index(drop=True, inplace=True)
hourly_data.to_csv('hourly_data.csv')
hourly_data.head(3)

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,CNTY_NM
0,2021-02-10 00:00:00,13.0,10.0,82.0,0.0,NaN,88.0,5.0,NaN,1015.1,NaN,NaN,Gonzales
1,2021-02-10 01:00:00,13.0,9.1,77.0,0.0,NaN,40.0,13.3,NaN,1015.6,NaN,NaN,Gonzales
2,2021-02-10 02:00:00,13.0,9.1,77.0,0.0,NaN,60.0,15.8,NaN,1016.1,NaN,NaN,Gonzales


# 2 Data Processing

## 2.1 Pick usefull features from outage data and weather data

Adding county information to weather data and pick usefull features from the **weather data**.

In [8]:
cnty.columns

Index(['X (Lat)', 'Y (Long)', 'CNTY_NM', 'CNTY_NBR', 'FIPS', 'Shape_Leng',
       'Shape_Area', 'County Location'],
      dtype='object')

In [9]:
hourly_data = pd.read_csv('hourly_data.csv') 
cnty_xycnty = cnty[['X (Lat)', 'Y (Long)', 'CNTY_NM', 'CNTY_NBR', 'FIPS']] 
weather_data = hourly_data.merge(cnty_xycnty, how='inner', on='CNTY_NM') #addX (Lat)', 'Y (Long) to weather data
weather_data['CNTY_NM'] = weather_data['CNTY_NM'].str.upper()
# weather_data['time'] = pd.Timestamp(weather_data['time'])
weather_data['time'] = pd.to_datetime(weather_data['time']) #str to timestamp
# df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
weather_data['State'] = 'Texas'
weather_cut = weather_data[['State', 'CNTY_NM', 'X (Lat)', 'Y (Long)', 'CNTY_NBR', 'FIPS', 'time', 'temp', 'coco']]

In [10]:
print(f'weather_cut shape: {weather_cut.shape}')
weather_cut.head(3)

weather_cut shape: (59760, 9)


,State,CNTY_NM,X (Lat),Y (Long),CNTY_NBR,FIPS,time,temp,coco
0,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 00:00:00,13.0,NaN
1,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 01:00:00,13.0,NaN
2,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 02:00:00,13.0,NaN


In [11]:
weather_cut['time'].unique()

array(['2021-02-10T00:00:00.000000000', '2021-02-10T01:00:00.000000000',
       '2021-02-10T02:00:00.000000000', '2021-02-10T03:00:00.000000000',
       '2021-02-10T04:00:00.000000000', '2021-02-10T05:00:00.000000000',
       '2021-02-10T06:00:00.000000000', '2021-02-10T07:00:00.000000000',
       '2021-02-10T08:00:00.000000000', '2021-02-10T09:00:00.000000000',
       '2021-02-10T10:00:00.000000000', '2021-02-10T11:00:00.000000000',
       '2021-02-10T12:00:00.000000000', '2021-02-10T13:00:00.000000000',
       '2021-02-10T14:00:00.000000000', '2021-02-10T15:00:00.000000000',
       '2021-02-10T16:00:00.000000000', '2021-02-10T17:00:00.000000000',
       '2021-02-10T18:00:00.000000000', '2021-02-10T19:00:00.000000000',
       '2021-02-10T20:00:00.000000000', '2021-02-10T21:00:00.000000000',
       '2021-02-10T22:00:00.000000000', '2021-02-10T23:00:00.000000000',
       '2021-02-11T00:00:00.000000000', '2021-02-11T01:00:00.000000000',
       '2021-02-11T02:00:00.000000000', '2021-02-11

<s>Drop the counties which didn't suffer outage.</s>

In [ ]:
# row_index_to_drop = []
# uniq_outage_county = outage['COUNTY'].unique()
# for i in range(len(weather_cut)):
#     if weather_cut['CNTY_NM'][i] not in uniq_outage_county:
#         row_index_to_drop.append(i)
        
# print(f'num of rows to drop in weather data: {len(row_index_to_drop)}')
# print(f'num of rows to be left: {len(weather_cut) - len(row_index_to_drop)}')

In [ ]:
# weather_cut_drop = weather_cut.drop(row_index_to_drop)
# weather_cut_drop.reset_index(drop=True, inplace=True)
# print(f'weather_cut_drop shape: {weather_cut_drop.shape}')
# weather_cut_drop.head(3)

Pick usefull features from the **outage data**.

In [12]:
outage_cut = outage.drop(['STATION LONG NAME', 'UNIT NAME', 'RESOURCE ENTITY'], axis=1)
outage_cut.rename(columns={'SEASONAL MAX MW (HSL)': 'MAX_MW', 'AVAILABLE MW AFTER OUTAGE/DERATE': 'AVAI_MW', 'MW REDUCTION FROM OUTAGE/DERATE': 'REDU_MW'}, inplace=True)
outage_cut.head(3)

,STATION,MAX_MW,AVAI_MW,REDU_MW,FUEL TYPE,START,END,COUNTY
0,NBOHR,197,20,177,WIND,2021-02-10 00:00:00,2021-02-12 04:20:00,GLASSCOCK
1,KEECHI,110,0,110,WIND,2021-02-10 00:30:00,2021-02-15 08:51:00,JACK
2,BLSUMIT3,13,1,12,WIND,2021-02-10 01:15:00,2021-02-10 18:46:00,HARDEMAN


## 2.2 Adding outage info to weather_cut_drop

For each county at each hour, checking if it's suffering outage or not. The `total MAX_MW, AVAIL_MW, REDU_MW` will be added. 

In [13]:
'''
Adding columns for reduced MW with different fuel type
'''
weather_cut_drop = weather_cut.copy()
# outage['FUEL TYPE'].unique():  ['WIND', 'NG', 'COAL', 'SOLAR', 'ESR', 'HYDRO', 'NUCLEAR']
rng = len(weather_cut_drop)
# WIND_recu_mw = [0] * rng
# NG_recu_mw = [0] * rng
# COAL_recu_mw = [0] * rng
# SOLAR_recu_mw = [0] * rng
# ESR_recu_mw = [0] * rng
# HYDRO_recu_mw = [0] * rng
# NUCLEAR_recu_mw = [0] * rng

weather_cut_drop = weather_cut.copy()

for FT in outage['FUEL TYPE'].unique():
    weather_cut_drop[FT + '_recu_mw'] = pd.DataFrame([0] * rng)
    
weather_cut_drop.head(3)

,State,CNTY_NM,X (Lat),Y (Long),CNTY_NBR,FIPS,time,temp,coco,WIND_recu_mw,NG_recu_mw,COAL_recu_mw,SOLAR_recu_mw,ESR_recu_mw,HYDRO_recu_mw,NUCLEAR_recu_mw
0,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 00:00:00,13.0,NaN,0,0,0,0,0,0,0
1,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 01:00:00,13.0,NaN,0,0,0,0,0,0,0
2,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 02:00:00,13.0,NaN,0,0,0,0,0,0,0


In [14]:
weather_cut_drop['time'].unique()

array(['2021-02-10T00:00:00.000000000', '2021-02-10T01:00:00.000000000',
       '2021-02-10T02:00:00.000000000', '2021-02-10T03:00:00.000000000',
       '2021-02-10T04:00:00.000000000', '2021-02-10T05:00:00.000000000',
       '2021-02-10T06:00:00.000000000', '2021-02-10T07:00:00.000000000',
       '2021-02-10T08:00:00.000000000', '2021-02-10T09:00:00.000000000',
       '2021-02-10T10:00:00.000000000', '2021-02-10T11:00:00.000000000',
       '2021-02-10T12:00:00.000000000', '2021-02-10T13:00:00.000000000',
       '2021-02-10T14:00:00.000000000', '2021-02-10T15:00:00.000000000',
       '2021-02-10T16:00:00.000000000', '2021-02-10T17:00:00.000000000',
       '2021-02-10T18:00:00.000000000', '2021-02-10T19:00:00.000000000',
       '2021-02-10T20:00:00.000000000', '2021-02-10T21:00:00.000000000',
       '2021-02-10T22:00:00.000000000', '2021-02-10T23:00:00.000000000',
       '2021-02-11T00:00:00.000000000', '2021-02-11T01:00:00.000000000',
       '2021-02-11T02:00:00.000000000', '2021-02-11

In [15]:
'''
loop through every pair
'''
rng = len(weather_cut_drop)
for index_weather in tqdm(range(rng)):
    for index_outage in range(len(outage_cut)):
        if weather_cut_drop['CNTY_NM'][index_weather] == outage_cut['COUNTY'][index_outage]: 
            #think aobut the efficiency on accesing data in dataframe
            if weather_cut_drop['time'][index_weather] > outage_cut['START'][index_outage] and \
                            weather_cut_drop['time'][index_weather] < outage_cut['END'][index_outage] :
                FT = outage_cut['FUEL TYPE'][index_outage]
                weather_cut_drop[FT + '_recu_mw'][index_weather] = \
                    weather_cut_drop[FT + '_recu_mw'][index_weather] + outage_cut['REDU_MW'][index_outage]
weather_cut_drop.to_csv('cnty_weather_out_v2.csv')
weather_cut_drop.head(3)

  1%|▋                                                                             | 482/59760 [00:09<20:11, 48.95it/s]<ipython-input-15-b1c5972c2f4f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_cut_drop[FT + '_recu_mw'][index_weather] = \
100%|████████████████████████████████████████████████████████████████████████████| 59760/59760 [20:57<00:00, 47.51it/s]


,State,CNTY_NM,X (Lat),Y (Long),CNTY_NBR,FIPS,time,temp,coco,WIND_recu_mw,NG_recu_mw,COAL_recu_mw,SOLAR_recu_mw,ESR_recu_mw,HYDRO_recu_mw,NUCLEAR_recu_mw
0,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 00:00:00,13.0,NaN,0,0,0,0,0,0,0
1,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 01:00:00,13.0,NaN,0,0,0,0,0,0,0
2,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 02:00:00,13.0,NaN,0,0,0,0,0,0,0


In [16]:
cnty_weather_out= pd.read_csv('cnty_weather_out_v2.csv')
cnty_weather_out.head(2)

,Unnamed: 0,State,CNTY_NM,X (Lat),Y (Long),CNTY_NBR,FIPS,time,temp,coco,WIND_recu_mw,NG_recu_mw,COAL_recu_mw,SOLAR_recu_mw,ESR_recu_mw,HYDRO_recu_mw,NUCLEAR_recu_mw
0,0,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 00:00:00,13.0,NaN,0,0,0,0,0,0,0
1,1,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 01:00:00,13.0,NaN,0,0,0,0,0,0,0


In [18]:
cnty_weather_out['Total_recu_mw'] = cnty_weather_out.iloc[:, -7:-1].sum(axis=1)

In [23]:
cnty_weather_out.head(2)

,Unnamed: 0,State,CNTY_NM,X (Lat),Y (Long),CNTY_NBR,FIPS,time,temp,coco,WIND_recu_mw,NG_recu_mw,COAL_recu_mw,SOLAR_recu_mw,ESR_recu_mw,HYDRO_recu_mw,NUCLEAR_recu_mw,Total_recu_mw
0,0,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 00:00:00,13.0,NaN,0,0,0,0,0,0,0,0
1,1,Texas,GONZALES,-97.492799,29.456415,90,48177,2021-02-10 01:00:00,13.0,NaN,0,0,0,0,0,0,0,0


In [24]:
cnty_weather_out.to_csv('cnty_weather_out_v2.csv')

In [ ]:
len(cnty_weather_out['CNTY_NM'].unique())

In [28]:
max(cnty_weather_out['Total_recu_mw'])

2461

In [29]:
min(cnty_weather_out['Total_recu_mw'])

0